In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

import sklearn
from sklearn.model_selection import train_test_split

In [25]:
import os
assert os.environ['COLAB_TPU_ADDR']

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [13]:
device = xm.xla_device()
# t1 = torch.ones(3, 3, device = dev)
# print(t1)

In [ ]:
!pip install SentencePiece transformers

In [11]:
df1 = pd.read_csv('/reddit_prepared_new.csv')
# df2 = pd.read_csv('/gab_prepared.csv')

In [12]:
df1.dropna(axis=0, inplace=True)
df1.reset_index(drop=True, inplace=True)

# df2.dropna(axis=0, inplace=True)
# df2.reset_index(drop=True, inplace=True)

# trainr, testr = train_test_split(df1, test_size=0.1, shuffle=True, random_state=754)
# train_textr, train_labelsr, test_textr, test_labelsr = trainr['text'], trainr['label'], testr['text'], testr['label']

# traing, testg = train_test_split(df2, test_size=0.1, shuffle=True, random_state=754)
# train_textg, train_labelsg, test_textg, test_labelsg = traing['text'], traing['label'], testg['text'], testg['label']

In [ ]:
rdf = pd.DataFrame(np.concatenate((np.array(train_textr).reshape(len(train_textr), 1), np.array(train_labelsr).reshape(len(train_labelsr), 1)), axis=1), columns=['text', 'label'])
gdf = pd.DataFrame(np.concatenate((np.array(train_textg).reshape(len(train_textg), 1), np.array(train_labelsg).reshape(len(train_labelsg), 1)), axis=1), columns=['text', 'label'])
df3 = pd.concat([rdf, gdf])
from sklearn.utils import shuffle
traindf = shuffle(df3)
traindf.reset_index(drop=True, inplace=True)
traindf.head()

,text,label
0,MeToo was just the continuation/mainstream ver...,0
1,.@SadiqKhan has been accused of misleading Lon...,0
2,Man charged with murder in deadly attack on gi...,0
3,Shes like 100 years old. So for her its not bad.,0
4,Every week I'm berated by some Christian for b...,0


In [ ]:
print(traindf.shape)
print(traindf['label'].value_counts())

(49279, 2)
0    31395
1    17884
Name: label, dtype: int64


In [ ]:
rdf = pd.DataFrame(np.concatenate((np.array(test_textr).reshape(len(test_textr), 1), np.array(test_labelsr).reshape(len(test_labelsr), 1)), axis=1), columns=['text', 'label'])
gdf = pd.DataFrame(np.concatenate((np.array(test_textg).reshape(len(test_textg), 1), np.array(test_labelsg).reshape(len(test_labelsg), 1)), axis=1), columns=['text', 'label'])

from sklearn.utils import shuffle
rdf.reset_index(drop=True, inplace=True)
gdf.reset_index(drop=True, inplace=True)

In [ ]:
print(rdf.shape)
print(rdf['label'].value_counts())

print(gdf.shape)
print(gdf['label'].value_counts())

(2221, 2)
0    1688
1     533
Name: label, dtype: int64
(3256, 2)
0    1826
1    1430
Name: label, dtype: int64


In [19]:
 tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
 
 def create_inputs(text):   
    inputs = [tokenizer(str(x), add_special_tokens=True, max_length=260, padding='max_length', truncation=True) for x in text ]
    # inputs
    input_ids = []
    attn_masks = []
    for input in inputs:
        input_ids.append(input['input_ids'])
        attn_masks.append(input['attention_mask'])

    return torch.tensor(input_ids), torch.tensor(attn_masks)
    # print(input_ids)
    # print(attn_masks)

In [ ]:
train_inputs, train_masks = create_inputs(traindf['text'])
train_labels = torch.tensor(traindf['label'])

rt_inputs, rt_masks = create_inputs(rdf['text'])
rt_labels = torch.tensor(rdf['label'])

gt_inputs, gt_masks = create_inputs(gdf['text'])
gt_labels = torch.tensor(gdf['label'])

In [ ]:
batch_size = 16

#preparing train data
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

#preparing test data
test_data = TensorDataset(rt_inputs, rt_masks, rt_labels)
test_sampler = RandomSampler(test_data)
rt_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

test_data = TensorDataset(gt_inputs, gt_masks, gt_labels)
test_sampler = RandomSampler(test_data)
gt_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
weights = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(traindf['label']), traindf['label'])
class_weights = torch.from_numpy(weights)
class_weights.to('cuda')
# print(weights)

tensor([0.7848, 1.3777], device='cuda:0', dtype=torch.float64)

In [ ]:
print(weights)
print(class_weights)
print(np.unique(traindf['label']))

[0.78482242 1.37773988]
tensor([0.7848, 1.3777], dtype=torch.float64)
[0 1]


## **Experiment #1: With XLNet-base-cased model**

In [15]:
def get_model(num_labels) :
    model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)
    model.to(device)
    return model

In [ ]:
model = get_model(num_labels=2)
# print(model)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
# special parameters for optimization

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}]

In [ ]:
# freezing all layers of the model except the last classification layer

# for param in model.roberta.parameters():
#     param.requires_grad = False

In [ ]:
# freezing all layers of the model except the last two hidden layers and the classification layer

# for param in model.roberta.encoder.layer[0:10].parameters():
#     param.requires_grad = False

In [ ]:
from transformers import AdamW
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [17]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
device = 'cuda'

In [ ]:
loss_fn = CrossEntropyLoss(weight=class_weights.float()).to('cuda')

In [18]:
def training( train_dataloader, epochs=5):

    train_loss_set = []

    # Number of training epochs (authors recommend between 2 and 4)
    epochs = epochs

    # trange is a tqdm wrapper around the normal python range
    for _ in range(epochs):
    
    
        # Training
        
        # Set our model to training mode (as opposed to evaluation mode)
        model.train()
        
        # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        
        # Train the df1 for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our df1loader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
            # loss = loss_fn(logits, b_labels)
            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            
            
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

        # print("Train loss: {}".format(tr_loss/nb_tr_steps))

In [20]:
def evaluation(validation_dataloader):

    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate df1 for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our df1loader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
        # Forward pass, calculate logit predictions
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = output[0]
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    # print("Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    return (eval_accuracy/nb_eval_steps)

In [21]:
def get_f1_score(test_dataloader):

    # Prediction on test set

    # Put model in evaluation mode
    model.eval()

    # Tracking variables 
    predictions , true_labels = [], []

    # Predict 
    for batch in test_dataloader:
    # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our df1loader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Store predictions and true labels
        predictions.append(logits)
        true_labels.append(label_ids)

    # calculating f1 score on test set 
    from sklearn.metrics import f1_score

    pred_list = []
    labels = []
    for i in true_labels:
        labels.extend(i)

    for i in range(len(predictions)):
        pred_list.append(np.argmax(predictions[i], axis=1).flatten())

    preds = []
    for i in pred_list:
        preds.extend(i)

    score = f1_score(labels, preds, average='weighted')
    # print ("F1 score: {}".format(score))
    return score, labels, preds

In [ ]:
training(train_dataloader, epochs=1)

Train loss: 0.20838743448499347


In [ ]:
np.mean([evaluation(rt_dataloader)*100, evaluation(gt_dataloader)*100])

Accuracy: 0.9145260989010988
Accuracy: 0.9207516339869282


91.76388664440135

In [ ]:
rt_score, labels, preds = get_f1_score(rt_dataloader)
gt_score, unk1, unk2 = get_f1_score(gt_dataloader)
np.mean([rt_score*100, gt_score*100])

F1 score: 0.9147233973536646
F1 score: 0.9206575037919573


91.76904505728109

In [ ]:
from sklearn.metrics import classification_report, plot_confusion_matrix
report = classification_report(labels, preds)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      1688
           1       0.82      0.83      0.82       533

    accuracy                           0.91      2221
   macro avg       0.88      0.89      0.88      2221
weighted avg       0.92      0.91      0.91      2221



In [ ]:
# plt.figure(figsize=(15,8))
# plt.title("Training loss")
# plt.xlabel("Batch")
# plt.ylabel("Loss")
# plt.plot(train_loss_set) #inside training function; hence not accessible 
# plt.show()

## Cross Validation

In [16]:
from sklearn.model_selection import StratifiedKFold

X = df1['text']
y = df1['label']

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y) 

print(skf)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


In [ ]:
acc_list = []
f1_score_list = []

for train_index, test_index in skf.split(X, y):
#   print("TRAIN:", train_index, "TEST:", test_index)
    train_text, test_text = X[train_index], X[test_index]
    train_label, test_label = y[train_index], y[test_index]
        
    train_inputs, train_masks = create_inputs(train_text)
    train_labels = torch.tensor(train_label.values)
    
    test_inputs, test_masks = create_inputs(test_text)
    test_labels = torch.tensor(test_label.values)
    
    batch_size = 16
    
    #preparing train data
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    
    #preparing test data
    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
    
    model = get_model(num_labels=2)
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}]
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=100, num_training_steps=(len(train_text)//32)*3)
    
    training(train_dataloader, epochs=3)
    
    acc = evaluation(test_dataloader)
    acc_list.append(acc)
    
    f1_score = get_f1_score(test_dataloader)[0]
    f1_score_list.append(f1_score)
    
    print(acc, f1_score)